# Rejection Rates

In [1]:
# Include necessary packages
library(tidyverse)

# library(mvtnorm)
library(condMVNorm)
library(MomTrunc)
# library(truncnorm)
# library(tmvtnorm)
# library(TTmoment)


library(doParallel)
# library(parallel)



path=getwd()
source(paste0(substring(path, 
                        1, 
                        tail(unlist(gregexpr(pattern ='/',path)),n=1)),
              "MixCenMVReg_EM.R"))

source(paste0(substring(path, 
                        1, 
                        tail(unlist(gregexpr(pattern ='/',path)),n=1)),
              "Util_Func.R"))

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.5     ✔ dplyr   1.0.7
✔ tidyr   1.1.4     ✔ stringr 1.4.0
✔ readr   2.0.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Loading required package: mvtnorm

Loading required package: foreach


Attaching package: ‘foreach’


The following objects are masked from ‘package:purrr’:

    accumulate, when


Loading required package: iterators

Loading required package: parallel



In [2]:
TrueDataGen=function(Replicate=30, n=1000,pie,beta,sigma){
    
N=n

G=length(pie) # Num of Clusters
P=dim(beta[[1]])[2] # Dimension of Y
D=dim(beta[[1]])[1] # Dimension of X (intercpt included)
    
out=list()
    for(i in 1:Replicate){
        df_X=list()
        for(g in 1:G){
            df_X[[g]]=cbind(rep(1,pie[g]*N),do.call('cbind',lapply(rep(pie[g]*N,1),rnorm,0)))
        }


        mu=list()
        for(g in 1:G){
            mu[[g]]=df_X[[g]]%*%beta[[g]]
        }


        df_Y=list()
        for(g in 1:G){
            df_Y[[g]]=as.data.frame(mu[[g]]+rmvnorm(pie[g]*N,mean=rep(0,P),sigma=sigma[[g]]))
            colnames(df_Y[[g]])=paste(rep('Y',P),1:P, sep = "", collapse = NULL)
            df_Y[[g]]$True_label=g
        }

        df_X=do.call('rbind',df_X)
        colnames(df_X)=paste(rep('X',D),1:D-1, sep = "", collapse = NULL)

        df_Y=do.call('rbind',df_Y)
        df_Labels=as.factor(df_Y$True_label)
        df_Y=as.matrix(df_Y[,1:P])


        df=list(df_Y,df_X,df_Labels)
        names(df)=c('Y','X','Labels')

        out[[i]]=df
    }
    
    return(out)
    
}

CensDataGen=function(True_Data,dl){
    Replicate=length(True_Data)

    for(r in 1:Replicate){
        df_Y=True_Data[[r]]$Y
        N=dim(df_Y)[1]
        P=dim(df_Y)[2]

        censorID=matrix(rep(0,N*P),nrow=N,ncol=P)
        colnames(censorID)=paste(rep('C',P),1:P, sep = "", collapse = NULL)
        
        for(i in 1:N){
            for(d in 1:P){
                if(df_Y[i,d]<=dl[[d]][1]){
                    df_Y[i,d]=dl[[d]][1]
                    censorID[i,d]=-1
                }else{
                    if(df_Y[i,d]>=dl[[d]][2]){
                        df_Y[i,d]=dl[[d]][2]
                        censorID[i,d]=1
                    }
                }
            }
        }
        True_Data[[r]]$Y=df_Y
        True_Data[[r]]$censorID=censorID
    }
        
    return(True_Data)
}
    
CensorRate=function(data){
    censor_rate=matrix(rep(NA,length(data)*2),nrow=length(data),ncol=2)

    for(i in 1:length(data)){
        censorID=data[[i]]$censorID
        censor_rate[i,]=c(sum(censorID[,1]!=0)/5000, sum(censorID[,2]!=0)/5000)

    }
    return(apply(censor_rate,2,mean))
}

# Generate data: multivariate Guassian mixture, G=3, d=2, moderately overlapping
set.seed(22)

BETA=list(matrix(c(2,0,20,-2),nrow=2,ncol=2),
          matrix(c(3,1,25,-3),nrow=2,ncol=2),
          matrix(c(3.5,2,30,-5),nrow=2,ncol=2))

SIGMA=list(matrix(c(1,0.1,0.1,1),nrow=2,ncol=2),
           matrix(c(2,0.2,0.2,0.5),nrow=2,ncol=2),
           matrix(c(0.5,0.3,0.3,2),nrow=2,ncol=2))                          
PIE=c(.1,.7,.2)

true_pars=list(PIE,BETA,SIGMA)
names(true_pars)=c('PIE','BETA','SIGMA')
save(true_pars,file = "true_pars.RData")

REP=500

data_true=TrueDataGen(REP,5000,PIE,BETA,SIGMA)
save(data_true,file = paste0("data_true_r",REP,"n5000.RData"))

DL=list(c(0,10),
        c(0,30))

data_cens_case1=CensDataGen(data_true,DL)
print(CensorRate(data_cens_case1))
save(data_cens_case1,file = paste0("data_cens_case1_r",REP,"n5000.RData"))

DL=list(c(2.5,10),
        c(0,26.5))

data_cens_case2=CensDataGen(data_true,DL)
print(CensorRate(data_cens_case2))
save(data_cens_case2,file = paste0("data_cens_case2_r",REP,"n5000.RData"))


DL=list(c(4,10),
        c(0,24))

data_cens_case3=CensDataGen(data_true,DL)
print(CensorRate(data_cens_case3))
save(data_cens_case3,file = paste0("data_cens_case3_r",REP,"n5000.RData"))

# options(repr.plot.width=16, repr.plot.height=8)


# hist(as.data.frame(data_true[[1]]$Y)$Y1,breaks=40, xlab = "True values of Y1", main="Distribution of true Y1")
# hist(as.data.frame(data_cens_case1[[1]]$Y)$Y1,breaks=20, xlim=c(-8,6),  xlab = "Observed values of Y1", main="Distribution of observed Y1")

# hist(data1$true$Y2,breaks=40, xlab = "True values of Y2", main="Distribution of true Y2")
# hist(data1$obs$Y2,breaks=20,  xlim=c(10,50), xlab = "Observed values of Y2", main="Distribution of observed Y2")


# ggplot(data1$obs, aes(x=Y1, y=Y2, color=True_label)) +
#   geom_point()

# ggplot(data1$obs, aes(x=X1, y=Y1, color=True_label)) +
#   geom_point()

# ggplot(data1$obs, aes(x=X1, y=Y2, color=True_label)) +
#   geom_point()

# save(data1,file = "data1.RData")

[1] 0.0417448 0.1368284
[1] 0.4037312 0.3693316
[1] 0.7195568 0.6180660


In [3]:
load(paste0("data_cens_case1_r",REP,"n5000.RData"))

load('true_pars.RData')

In [4]:
start=Sys.time()

no_cores=detectCores(logical = TRUE)  
# returns the number of available hardware threads, 
# and if it is FALSE, returns the number of physical cores
# print(no_cores)

cl=makeCluster(no_cores)  
registerDoParallel(cl)  

# replicate=length(data_cens_case1)
replicate=2

case1_cens_RR=foreach(i=1:replicate, .errorhandling="remove") %dopar% {
  
    library(condMVNorm)
    library(MomTrunc)
    MixCenMVReg_EM(Y=data_cens_case1[[i]]$Y,
                   X=data_cens_case1[[i]]$X,
                   C=data_cens_case1[[i]]$censorID, 
                   G=3, pie_hat=true_pars$PIE, beta_hat=true_pars$BETA, sigma_hat=true_pars$SIGMA,
                   Max.iter=5000, diff.tol=1e-4, print=FALSE)
}
stopCluster(cl)
# save(model1.1,file = "model1.1.RData")
print(Sys.time()-start)

save(case1_cens_RR,file=paste0("case1_cens_RR_r",replicate,"n5000.RData"))


Time difference of 3.392924 mins


In [5]:
str(case1_cens_RR)

List of 2
 $ :List of 12
  ..$ Iterations: num 39
  ..$ Converged : logi TRUE
  ..$ LogLik    : num -16716
  ..$ AIC       : num 33477
  ..$ BIC       : num 33627
  ..$ ICL       : num 34303
  ..$ Pie       : Named num [1:3] 0.0992 0.702 0.1987
  .. ..- attr(*, "names")= chr [1:3] "pie1" "pie2" "pie3"
  ..$ Beta      :List of 3
  .. ..$ beta1: num [1:2, 1:2] 1.9905 -0.0324 20.033 -1.9489
  .. .. ..- attr(*, "dimnames")=List of 2
  .. .. .. ..$ : chr [1:2] "X0" "X1"
  .. .. .. ..$ : chr [1:2] "Y1" "Y2"
  .. ..$ beta2: num [1:2, 1:2] 2.96 1 24.99 -2.99
  .. .. ..- attr(*, "dimnames")=List of 2
  .. .. .. ..$ : chr [1:2] "X0" "X1"
  .. .. .. ..$ : chr [1:2] "Y1" "Y2"
  .. ..$ beta3: num [1:2, 1:2] 3.49 1.99 29.99 -4.92
  .. .. ..- attr(*, "dimnames")=List of 2
  .. .. .. ..$ : chr [1:2] "X0" "X1"
  .. .. .. ..$ : chr [1:2] "Y1" "Y2"
  ..$ Sigma     :List of 3
  .. ..$ sigma1: num [1:2, 1:2] 1.071 0.123 0.123 1.006
  .. .. ..- attr(*, "dimnames")=List of 2
  .. .. .. ..$ : chr [1:2] "Y1" "